QUESTION: 1

In [21]:
import numpy as np

In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras import backend as K

In [23]:
from subprocess import check_output

In [4]:
train = pd.read_csv("Train.csv") 
test = pd.read_csv("Test.csv")
test1 = test

In [5]:
train.head()

people_ID        Region  Gender Designation             Name Married  \
0          1  Bhubaneshwar  Female         Mrs            mansi     YES   
1          2  Bhubaneshwar  Female         Mrs        riya masi     YES   
2          3  Bhubaneshwar  Female         Mrs           sunita      NO   
3          4  Bhubaneshwar  Female         Mrs   anjali @ babli     YES   
4          5  Bhubaneshwar  Female         Mrs  champa karketta      NO   

   Children     Occupation Mode_transport  cases/1M  ...    HBB d-dimer  \
0       1.0         Farmer         Public         2  ...   93.0   233.0   
1       2.0         Farmer           Walk         2  ...   56.0   328.0   
2       1.0        Cleaner         Public         2  ...  137.0   213.0   
3       1.0         Driver            Car         2  ...  167.0   275.0   
4       2.0  Manufacturing            Car         2  ...  153.0   331.0   

   Heart rate  HDL cholesterol Charlson Index Blood Glucose  Insurance  \
0        82.0             58.0             27             7  3600000.0   
1        89.0             68.0              5             6  1600000.0   
2        77.0             43.0             40             6  3400000.0   
3        64.0             60.0             27             7   700000.0   
4        71.0             64.0             32             7  3200000.0   

    salary  FT/month  Infect_Prob  
0  1300000       2.0    49.135010  
1   400000       1.0    51.147880  
2   900000       1.0    73.224000  
3  2300000       1.0    48.779225  
4  1100000       1.0    87.868800  

[5 rows x 28 columns]

In [6]:
test.head()

people_ID Region  Gender Designation                Name Married  Children  \
0       5942  Delhi  Female         Mrs  smt rekha prajapat     YES         2   
1      18664  Delhi    Male          Mr              nirmal     YES         2   
2       5603  Delhi  Female         Mrs               pinky     YES         2   
3       5649  Delhi  Female         Mrs       pooja @aafrin     YES         2   
4       5099  Delhi  Female         Mrs              anjali     YES         2   

  Occupation Mode_transport  cases/1M  ...  Platelets  HBB  d-dimer  \
0     Driver         Public         4  ...        153  196      240   
1      Legal           Walk         4  ...         95  138      241   
2      Sales            Car         4  ...         40  166      236   
3      Sales            Car         4  ...         78   83      211   
4   Business            Car         4  ...        109  207      312   

   Heart rate HDL cholesterol Charlson Index  Blood Glucose  Insurance  \
0          85              53             17              3    3900000   
1          81              61              2              5    1800000   
2          88              47             24              3    5000000   
3          87              52             13              6    3100000   
4          94              68             39              5    2300000   

    salary  FT/month  
0  1300000         1  
1  1300000         1  
2  2000000         2  
3   600000         2  
4  1500000         1  

[5 rows x 27 columns]

In [7]:
train.drop(['Name'],1,inplace=True)
train.fillna(train.mean(), inplace=True)
train[train.isnull().any(axis=1)]
cleanup_nums = {"Region":     {"Bhubaneshwar": 1, "Chandigarh": 2, "Chennai": 3,"Delhi": 4,"Bengaluru": 5,}, "Gender":    {"Male": 1,"Female": 2},
               "Designation": {"Mrs": 1,"Mr": 2},"Married": {"YES": 1,"NO": 2},"Occupation": {"Business": 9,"Researcher": 1,"Cleaner": 2,"Sales": 3,"Farmer": 4,"Driver": 5,"Clerk": 6,"Legal": 7,"Manufacturing": 8},
               "Mode_transport": {"Walk": 0,"Public": 1,"Car": 2},"comorbidity": {"None":4,"Hypertension": 1,"Diabetes": 2,"Coronary Heart Disease": 3},
               "Pulmonary score": {"<400": 1,"<100": 2,"<300": 3,"<200": 4},"cardiological pressure": {"Normal": 4,"Stage-01": 1,"Stage-02": 2,"Elevated": 3}}
train.replace(cleanup_nums, inplace=True)

train.head()


people_ID  Region  Gender  Designation  Married  Children  Occupation  \
0          1       1       2            1        1       1.0         4.0   
1          2       1       2            1        1       2.0         4.0   
2          3       1       2            1        2       1.0         2.0   
3          4       1       2            1        1       1.0         5.0   
4          5       1       2            1        2       2.0         8.0   

   Mode_transport  cases/1M  Deaths/1M  ...    HBB  d-dimer  Heart rate  \
0             1.0         2          0  ...   93.0    233.0        82.0   
1             0.0         2          0  ...   56.0    328.0        89.0   
2             1.0         2          0  ...  137.0    213.0        77.0   
3             2.0         2          0  ...  167.0    275.0        64.0   
4             2.0         2          0  ...  153.0    331.0        71.0   

   HDL cholesterol  Charlson Index  Blood Glucose  Insurance   salary  \
0             58.0              27              7  3600000.0  1300000   
1             68.0               5              6  1600000.0   400000   
2             43.0              40              6  3400000.0   900000   
3             60.0              27              7   700000.0  2300000   
4             64.0              32              7  3200000.0  1100000   

   FT/month  Infect_Prob  
0       2.0    49.135010  
1       1.0    51.147880  
2       1.0    73.224000  
3       1.0    48.779225  
4       1.0    87.868800  

[5 rows x 27 columns]

In [8]:
target_column = ['Infect_Prob'] 
predictors = list(set(list(train.columns))-set(target_column))
train[predictors] = train[predictors]/train[predictors].max()
train.describe()

people_ID        Region        Gender   Designation       Married  \
count  10714.000000  10714.000000  10714.000000  10714.000000  10714.000000   
mean       0.444366      0.562087      0.753080      0.746920      0.604209   
std        0.313226      0.293544      0.249993      0.249993      0.203098   
min        0.000044      0.200000      0.500000      0.500000      0.500000   
25%        0.118055      0.400000      0.500000      0.500000      0.500000   
50%        0.440780      0.600000      1.000000      0.500000      0.500000   
75%        0.667184      0.800000      1.000000      1.000000      0.500000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

           Children   Occupation  Mode_transport      cases/1M     Deaths/1M  \
count  10714.000000  9967.000000    10711.000000  10714.000000  10714.000000   
mean       0.531385     0.553660        0.499393      0.410736      0.261620   
std        0.403129     0.286315        0.407762      0.340159      0.427152   
min        0.000000     0.111111        0.000000      0.125000      0.000000   
25%        0.000000     0.333333        0.000000      0.250000      0.000000   
50%        0.500000     0.555556        0.500000      0.250000      0.000000   
75%        1.000000     0.777778        1.000000      0.500000      0.500000   
max        1.000000     1.000000        1.000000      1.000000      1.000000   

       ...           HBB       d-dimer    Heart rate  HDL cholesterol  \
count  ...  10714.000000  10714.000000  10714.000000     10714.000000   
mean   ...      0.553195      0.786549      0.748474         0.751896   
std    ...      0.262560      0.118105      0.140748         0.148818   
min    ...      0.095238      0.571429      0.500000         0.500000   
25%    ...      0.323810      0.691429      0.630000         0.628571   
50%    ...      0.553195      0.786549      0.748474         0.757143   
75%    ...      0.785714      0.882857      0.860000         0.885714   
max    ...      1.000000      1.000000      1.000000         1.000000   

       Charlson Index  Blood Glucose     Insurance        salary  \
count    10714.000000   10714.000000  10714.000000  10714.000000   
mean         0.500702       0.716432      0.528984      0.567267   
std          0.295711       0.199967      0.262217      0.264172   
min          0.000000       0.428571      0.060000      0.130435   
25%          0.250000       0.571429      0.320000      0.347826   
50%          0.500000       0.714286      0.528984      0.565217   
75%          0.750000       0.857143      0.740000      0.782609   
max          1.000000       1.000000      1.000000      1.000000   

           FT/month   Infect_Prob  
count  10714.000000  10714.000000  
mean       0.502302     52.280680  
std        0.393633     10.674667  
min        0.000000     29.289600  
25%        0.000000     47.277426  
50%        0.500000     49.008389  
75%        1.000000     52.762313  
max        1.000000     97.632000  

[8 rows x 27 columns]

In [17]:
from keras.layers import Activation, Dense
model = Sequential()
model.add(Dense(500, input_dim=26, activation= "relu"))
model.add(Dense(100, activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1))

In [18]:
from sklearn.model_selection import train_test_split
X_train = train[predictors].values
y_train = train[target_column].values

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
#print(X_train.shape); print(X_test.shape)

In [19]:
model.compile(loss='mean_squared_error' , optimizer="adam", metrics=["mean_squared_error"])

In [20]:
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
10714/10714 [==============================] - 7s 623us/step - loss: nan - mean_squared_error: nan
Epoch 2/50
10714/10714 [==============================] - 4s 344us/step - loss: nan - mean_squared_error: nan
Epoch 3/50
10714/10714 [==============================] - 4s 365us/step - loss: nan - mean_squared_error: nan
Epoch 4/50
10714/10714 [==============================] - 4s 382us/step - loss: nan - mean_squared_error: nan
Epoch 5/50
10714/10714 [==============================] - 4s 332us/step - loss: nan - mean_squared_error: nan0s - loss: nan - m
Epoch 6/50
10714/10714 [==============================] - 4s 346us/step - loss: nan - mean_squared_error: nan
Epoch 7/50
10714/10714 [==============================] - 5s 427us/step - loss: nan - mean_squared_error: nan
Epoch 8/50
10714/10714 [==============================] - 5s 466us/step - loss: nan - mean_squared_error: nan
Epoch 9/50
10714/10714 [==============================] - 4s 377us/step - loss: nan - mean_squared_err

In [158]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy {0}'.format(score[1]))
print('Test loss:', score[0])

Test accuracy 84.65308380126953
Test loss: 84.65307933042061


In [175]:
test.drop(['Name'],1,inplace=True)
test.fillna(test.mean(), inplace=True)
test[test.isnull().any(axis=1)]
cleanup_nums = {"Region":     {"Bhubaneshwar": 1, "Chandigarh": 2, "Chennai": 3,"Delhi": 4,"Bengaluru": 5, "Hyderabad": 6,"Kolkata": 7,"Mumbai": 8,"Pune": 9,"Thiruvananthapuram": 10}, "Gender":    {"Male": 1,"Female": 2},
               "Designation": {"Mrs": 1,"Mr": 2},"Married": {"YES": 1,"NO": 2},"Occupation": {"Business": 9,"Researcher": 1,"Cleaner": 2,"Sales": 3,"Farmer": 4,"Driver": 5,"Clerk": 6,"Legal": 7,"Manufacturing": 8},
               "Mode_transport": {"Walk": 0,"Public": 1,"Car": 2},"comorbidity": {"None":4,"Hypertension": 1,"Diabetes": 2,"Coronary Heart Disease": 3},
               "Pulmonary score": {"<400": 1,"<100": 2,"<300": 3,"<200": 4},"cardiological pressure": {"Normal": 4,"Stage-01": 1,"Stage-02": 2,"Elevated": 3}}
test.replace(cleanup_nums, inplace=True)

test.head()

people_ID  Region  Gender  Designation  Married  Children  Occupation  \
0       5942       4       2            1        1         2           5   
1      18664       4       1            2        1         2           7   
2       5603       4       2            1        1         2           3   
3       5649       4       2            1        1         2           3   
4       5099       4       2            1        1         2           9   

   Mode_transport  cases/1M  Deaths/1M  ...  Platelets  HBB  d-dimer  \
0               1         4          1  ...        153  196      240   
1               0         4          1  ...         95  138      241   
2               2         4          1  ...         40  166      236   
3               2         4          1  ...         78   83      211   
4               2         4          1  ...        109  207      312   

   Heart rate  HDL cholesterol  Charlson Index  Blood Glucose  Insurance  \
0          85               53              17              3    3900000   
1          81               61               2              5    1800000   
2          88               47              24              3    5000000   
3          87               52              13              6    3100000   
4          94               68              39              5    2300000   

    salary  FT/month  
0  1300000         1  
1  1300000         1  
2  2000000         2  
3   600000         2  
4  1500000         1  

[5 rows x 26 columns]

In [128]:
null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum()

Series([], dtype: float64)

In [127]:
print(train[train.isnull().any(axis=1)][null_columns].head())

Empty DataFrame
Columns: []
Index: []


In [120]:
train.fillna(train.mean(), inplace=True)

In [121]:
print(train[train.isnull().any(axis=1)][null_columns].head())

Empty DataFrame
Columns: [Occupation, Mode_transport, comorbidity, cardiological pressure]
Index: []


In [122]:
null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum()

Series([], dtype: float64)

In [180]:
predictors = list(set(list(test.columns))-set(target_column))
test[predictors] = test[predictors]/test[predictors].max()
test.describe()


people_ID        Region        Gender   Designation       Married  \
count  14498.000000  14498.000000  14498.000000  14498.000000  14498.000000   
mean       0.573932      0.743427      0.745068      0.754932      0.603842   
std        0.270781      0.191979      0.249960      0.249960      0.202832   
min        0.154173      0.400000      0.500000      0.500000      0.500000   
25%        0.307284      0.600000      0.500000      0.500000      0.500000   
50%        0.653161      0.800000      0.500000      1.000000      0.500000   
75%        0.805797      0.900000      1.000000      1.000000      0.500000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

           Children    Occupation  Mode_transport      cases/1M     Deaths/1M  \
count  14498.000000  14498.000000    14498.000000  14498.000000  14498.000000   
mean       0.489481      0.551218        0.501586      0.452826      0.447579   
std        0.405917      0.287989        0.409286      0.357825      0.317855   
min        0.000000      0.111111        0.000000      0.000000      0.000000   
25%        0.000000      0.333333        0.000000      0.230769      0.333333   
50%        0.500000      0.555556        0.500000      0.307692      0.333333   
75%        1.000000      0.777778        1.000000      0.846154      0.666667   
max        1.000000      1.000000        1.000000      1.000000      1.000000   

       ...     Platelets           HBB       d-dimer    Heart rate  \
count  ...  14498.000000  14498.000000  14498.000000  14498.000000   
mean   ...      0.543040      0.543252      0.784764      0.750217   
std    ...      0.263715      0.262895      0.124875      0.145497   
min    ...      0.093750      0.095238      0.571429      0.500000   
25%    ...      0.318750      0.314286      0.677143      0.630000   
50%    ...      0.543750      0.538095      0.782857      0.750000   
75%    ...      0.775000      0.771429      0.891429      0.880000   
max    ...      1.000000      1.000000      1.000000      1.000000   

       HDL cholesterol  Charlson Index  Blood Glucose     Insurance  \
count     14498.000000    14498.000000   14498.000000  14498.000000   
mean          0.748827        0.499936       0.713675      0.530970   
std           0.147732        0.297196       0.202329      0.275077   
min           0.500000        0.000000       0.428571      0.060000   
25%           0.614286        0.250000       0.571429      0.300000   
50%           0.742857        0.500000       0.714286      0.540000   
75%           0.871429        0.750000       0.857143      0.760000   
max           1.000000        1.000000       1.000000      1.000000   

             salary      FT/month  
count  14498.000000  14498.000000  
mean       0.565226      0.501483  
std        0.263303      0.409476  
min        0.130435      0.000000  
25%        0.347826      0.000000  
50%        0.565217      0.500000  
75%        0.782609      1.000000  
max        1.000000      1.000000  

[8 rows x 26 columns]

In [181]:
pred2=model.predict(test)

In [178]:
train[['Infect_Prob'] ]

Infect_Prob
0        49.135010
1        51.147880
2        73.224000
3        48.779225
4        87.868800
...            ...
10709    41.981760
10710    47.148663
10711    47.708533
10712    53.803565
10713    68.342400

[10714 rows x 1 columns]

In [183]:
pred2

array([[53.28588 ],
       [55.499874],
       [59.77737 ],
       ...,
       [46.48702 ],
       [60.02596 ],
       [41.556797]], dtype=float32)

In [187]:
out_df = pd.DataFrame({'people_ID':test1['people_ID'].values})
out_df .insert(loc=1, column='Infect_Prob', value=pred2)
out_df.to_csv('submission.csv', index=False)